# Gaussian pulse evokes travelling waves in 1D SNN

- https://www.nature.com/articles/ncomms4675#Sec24

- https://www.nature.com/articles/ncomms4675.pdf

- https://static-content.springer.com/esm/art%3A10.1038%2Fncomms4675/MediaObjects/41467_2014_BFncomms4675_MOESM1276_ESM.pdf

In [1]:
# fileName

fileName = 'eg004_1D_evocked_waves'

In [2]:
# set libs

import pyNN.spiNNaker as sim
import numpy as np
import matplotlib.pyplot as plt

Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation setting

dt         = 1          # (ms) #0.1
simtime    = 200       # (ms)
sim.setup(timestep = dt, 
          min_delay = 1,
          max_delay = 14) 


2021-04-10 12:49:04 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2021-04-10 12:49:04 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2021-04-10 12:49:04 WARNING: /home/spinnaker/CNS-tutorial/reports has 71 old reports that have not been closed
2021-04-10 12:49:04 WARNING: /home/spinnaker/CNS-tutorial/application_generated_data_files has 72 old reports that have not been closed
2021-04-10 12:49:04 INFO: Setting time scale factor to 1.
2021-04-10 12:49:04 INFO: Setting machine time step to 1000 micro-seconds.


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


0

In [19]:
# making the network 
from pyNN import space 

n_cells = {}
n_cells['Exc'] = 1000
n_cells['Inh'] = 1000


pops = {}
for pop in ['Exc', 'Inh']:
    pops[pop] = sim.Population(n_cells[pop], # number of cells
                           sim.IF_cond_exp, # cell model
                           sim.IF_cond_exp.default_parameters, # cell parameters
                           structure = space.Line(dx=1.0, 
                                                  x0=0.0, 
                                                  y=0.0, 
                                                  z=0.0),
                           label=pop)

    pops[pop].record(['spikes', 'v', 'gsyn_exc', 'gsyn_inh'])



In [ ]:
# compute manual list of connections

do_run = False
    
if do_run:

    def compute_manual_list(n_cells_i, n_cells_j, weights, d_thresh, p_thresh, width): 
            v_c = 0.35 # m/s
            d0 = 1 #ms
            scale = 1

            connections = {}
            probabilities = {}
            distances = {}

            connections = []
            probabilities = []
            distances = []
            for pre in range(n_cells_i):
                for post in range(n_cells_j):
                    d_ij = np.sqrt((pre - post)**2)
                    delay = d0 + d_ij / v_c
                    if d_ij > d_thresh: 
                        distances.append(d_ij)
                        p_ij = scale*np.exp(-0.5 * (d_ij**2/width**2))
                        probabilities.append(p_ij)

                        if p_ij > p_thresh:
                            connections.append([pre, post, weights, delay])#, [d_ij, p_ij]])

            return connections, distances, probabilities

    # computation step
    weight = {('Exc', 'Exc') : 0.06,
              ('Exc', 'Inh') : 0.06,
              ('Inh', 'Inh') : 0.018,
              ('Inh', 'Exc') : 0.018,
              }

    d_thresh = {('Exc', 'Exc') : 0.0,
              ('Exc', 'Inh') : 0.0,
              ('Inh', 'Inh') : 0.0,
              ('Inh', 'Exc') : 0.0,
              }


    p_thresh = {('Exc', 'Exc') : 0.01,
              ('Exc', 'Inh') : 0.01,
              ('Inh', 'Inh') : 0.01,
              ('Inh', 'Exc') : 0.01,
              }


    width = {('Exc', 'Exc') : 1,
              ('Exc', 'Inh') : 1,
              ('Inh', 'Inh') : 1,
              ('Inh', 'Exc') : 1,
              }


    connections = {}
    distances = {}
    probabilities = {}

    for i in ['Exc', 'Inh']:
        for j in ['Exc', 'Inh']:
            connections[i,j], distances[i,j], probabilities[i,j] = compute_manual_list(n_cells[i], 
                                                                               n_cells[j], 
                                                                               weight[i, j], 
                                                                               d_thresh[i, j], 
                                                                               p_thresh[i, j], 
                                                                               width[i, j])
    do_run=True
    if do_run:
        # visual check, e.g. with Exc > Exc
        fig, ax = plt.subplots(1,2, figsize=(11,5))
        fig.tight_layout(pad=3)
        axes_list = fig.axes

        axes_list[0].plot(np.asarray(connections['Exc', 'Exc']).T[0], np.asarray(connections['Exc', 'Exc']).T[1],'go', label='Exc (i) - Exc (j)')
        axes_list[0].grid()
        axes_list[0].legend()
        axes_list[0].set_title('scatter plot of connections')
        axes_list[0].set_xlabel('i cells')
        axes_list[0].set_ylabel('j cells')
        axes_list[0].set_xlim(20,40)
        axes_list[0].set_ylim(20,40)

        axes_list[1].plot(distances['Exc', 'Exc'],probabilities['Exc', 'Exc'],'g+')
        axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Exc', 'Exc']**2)), 'g:')
        axes_list[1].grid()
        axes_list[1].set_xlim(0,50)
        axes_list[1].set_ylim(0,1)
        axes_list[1].axhline(p_thresh['Exc', 'Exc'], color='k', label='p_threshold', )
        axes_list[1].set_title('probability as function of distance')
        axes_list[1].set_xlabel('distance')
        axes_list[1].set_ylabel('probability')
        axes_list[1].legend()


In [7]:
# make projection
projs = {}

do_run = False
if do_run:
    receptors = {}
    receptors['Exc', 'Exc'] = 'excitatory'
    receptors['Exc', 'Inh'] = 'excitatory'
    receptors['Inh', 'Inh'] = 'inhibitory'
    receptors['Inh', 'Exc'] = 'inhibitory'

    projs = {}
    for i in ['Exc', 'Inh']:
        for j in ['Exc', 'Inh']:
            projs[i,j] = sim.Projection(
                                    pops[i],
                                    pops[j],
                                    connector = sim.FromListConnector(connections[i,j]), 
                                    receptor_type = receptors[i,j],
                                    space = space.Space(axes = 'x'),            
                                    label = i + ' - ' + j,
                                )
            
do_run = True
if do_run:
    receptors = {}
    receptors['Exc', 'Exc'] = 'excitatory'
    receptors['Exc', 'Inh'] = 'excitatory'
    receptors['Inh', 'Inh'] = 'inhibitory'
    receptors['Inh', 'Exc'] = 'inhibitory'

    projs = {}
    l = {}
    for i in ['Exc', 'Inh']:
        for j in ['Exc', 'Inh']:

            projs[i,j] = sim.Projection(
                                    pops[i],
                                    pops[j],
                                    connector = sim.FixedProbabilityConnector(0.1), 
                                    receptor_type = receptors[i,j],
                                    synapse_type = sim.StaticSynapse(weight=0.05, delay=1),
                                    space = space.Space(axes = 'x'),            
                                    label = i + ' - ' + j,
                                )

In [8]:
# make the thalamic gaussian pulse

popName = 'Thalamus'  
tot_cells = n_cells['Exc']

spike_times = [[]]*(tot_cells) #list of spike lists, where one spike list is related to one spike source
random_sources_idx = [np.random.randint(tot_cells*0.40, tot_cells*0.60) for i in range(tot_cells)]

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = [abs(np.random.normal(loc=50, scale=5)) for n in range(5)]


pops[popName] = sim.Population(tot_cells, 
                               sim.SpikeSourceArray(spike_times),
                               structure = space.Line(dx=1.0, 
                                              x0=0.0, 
                                              y=0.0, 
                                              z=0.0),
                               
                               label = popName)

pops[popName].record('spikes')

do_plot = True
if do_plot:
    fig, axes = plt.subplots(1,1)
    axes_list = fig.axes
    axes_list[0].eventplot(spike_times)
    axes_list[0].set_xlabel('[ms]')
    axes_list[0].set_ylabel('thalamic spike sources')
    axes_list[0].set_title('rasterplot')
    axes_list[0].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# make the thalamic projections

projs['Thalamus', 'Exc'] = sim.Projection(pops['Thalamus'], 
                                       pops['Exc'],
                                       connector = sim.OneToOneConnector(),
                                       synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                       receptor_type = 'excitatory',
                                       space = space.Space(axes = 'x'),                                                     
                                       label = 'Thalamus - ' + 'Exc')

In [10]:
# run simulation

sim.run(simtime) # simtime=500 [ms]

2021-04-10 12:42:48 INFO: Simulating for 200 1.0ms timesteps using a hardware timestep of 1000us
2021-04-10 12:42:48 INFO: Starting execution process
2021-04-10 12:42:51 INFO: Time 0:00:03.448795 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2021-04-10 12:43:01 INFO: Time 0:00:09.628624 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2021-04-10 12:43:07 INFO: Time 0:00:05.535046 taken by PartitionAndPlacePartitioner
Created spalloc job 5926991
2021-04-10 12:43:07 INFO: Created spalloc job 5926991
Waiting for board power commands to complete.
2021-04-10 12:43:07 INFO: Waiting for board power commands to complete.
2021-04-10 12:43:12 INFO: Time 0:00:05.049986 taken by SpallocAllocator
2021-04-10 1

200.0

In [11]:
# save the results
outputs = {}

for layer in ['Exc', 'Inh']:
    outputs[layer] = pops[layer].get_data()
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[layer].write_data(fileName + '_' + str(layer) + '_' + str(recording) + '.pkl')

Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%  

In [12]:
def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

In [13]:
# check results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh
v
gsyn_exc
gsyn_inh


dict_keys([('Exc', 'v'), ('Exc', 'gsyn_exc'), ('Exc', 'gsyn_inh'), ('Exc', 'spikes'), ('Inh', 'v'), ('Inh', 'gsyn_exc'), ('Inh', 'gsyn_inh'), ('Inh', 'spikes')])

In [14]:
# check the state variable

fig, axes = plt.subplots(1, 2)#, figsize=(5,9))
fig.tight_layout(pad=5)
fig.suptitle('rasterplot and voltage plot')
axes_list = fig.axes

for idx, value in enumerate(['Exc', 'Inh']):
    axes_list[idx].eventplot(results[value, 'spikes'], colors='r')
    imv = axes_list[idx].imshow(results[value, 'v'].T)
    axes_list[idx].set_title(str(value) + ' layer')
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')
    axes_list[idx].set_xlim(0, simtime)
    fig.colorbar(imv, ax=axes_list[idx], fraction=0.020, label='[mV]')
    



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
#results['Exc', 'spikes']

In [18]:
# check the conductances

for layer in ['Exc', 'Inh']:    
    fig, axes = plt.subplots(1, 2)
    fig.tight_layout(pad=5)
    axes_list = fig.axes
    fig.suptitle('rasterplot and gsyn plot of ' + layer + 'layer')
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):

        #axes_list[idx].eventplot(results[value, 'spikes'], colors='r')
        im = axes_list[idx].imshow(results[layer, gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        axes_list[idx].set_xlim(0, simtime)
        fig.colorbar(im, ax=axes_list[idx], fraction=0.020, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#.get(['source', 'target', 'weight', 'delay'], format='list')
projs

{('Exc', 'Exc'): projection Exc - Exc,
 ('Exc', 'Inh'): projection Exc - Inh,
 ('Inh', 'Exc'): projection Inh - Exc,
 ('Inh', 'Inh'): projection Inh - Inh,
 ('Thalamus', 'Exc'): projection Thalamus - Exc}

In [19]:
projs['Exc', 'Inh'].get(['source', 'target', 'weight', 'delay'], format='list')[0:100]

2021-04-10 11:02:30 WARNING: Ignoring request to get source as with_address=True. 
2021-04-10 11:02:30 WARNING: Ignoring request to get target as with_address=True. 
Getting ['source', 'target', 'weight', 'delay']s for projection between Exc and Inh
|0%                          50%                         100%|


array([( 0,  1, 0.05999756,  4.), ( 0,  2, 0.05999756,  7.),
       ( 0,  3, 0.05999756, 10.), ( 1,  0, 0.05999756,  4.),
       ( 1,  2, 0.05999756,  4.), ( 1,  3, 0.05999756,  7.),
       ( 1,  4, 0.05999756, 10.), ( 2,  0, 0.05999756,  7.),
       ( 2,  1, 0.05999756,  4.), ( 2,  3, 0.05999756,  4.),
       ( 2,  4, 0.05999756,  7.), ( 2,  5, 0.05999756, 10.),
       ( 3,  0, 0.05999756, 10.), ( 3,  1, 0.05999756,  7.),
       ( 3,  2, 0.05999756,  4.), ( 3,  4, 0.05999756,  4.),
       ( 3,  5, 0.05999756,  7.), ( 3,  6, 0.05999756, 10.),
       ( 4,  1, 0.05999756, 10.), ( 4,  2, 0.05999756,  7.),
       ( 4,  3, 0.05999756,  4.), ( 4,  5, 0.05999756,  4.),
       ( 4,  6, 0.05999756,  7.), ( 4,  7, 0.05999756, 10.),
       ( 5,  2, 0.05999756, 10.), ( 5,  3, 0.05999756,  7.),
       ( 5,  4, 0.05999756,  4.), ( 5,  6, 0.05999756,  4.),
       ( 5,  7, 0.05999756,  7.), ( 5,  8, 0.05999756, 10.),
       ( 6,  3, 0.05999756, 10.), ( 6,  4, 0.05999756,  7.),
       ( 6,  5, 0.059997